<a href="https://colab.research.google.com/github/kaustubhgupta/pandas-nvidia-rapids/blob/main/GPU_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Rapids Setup

In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

In [ ]:
# This will crash the session once
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

In [ ]:
# This will restart your kernel one last time
import condacolab
condacolab.install()

In [1]:
import condacolab
condacolab.check()

✨🍰✨ Everything looks OK!


In [ ]:
 # 15 minutes process
 !python rapidsai-csp-utils/colab/install_rapids.py stable

## Model Data prep

In [1]:
import cudf
from cuml import make_regression, train_test_split
from cuml.linear_model import LinearRegression as cuLinearRegression
from cuml.metrics.regression import r2_score
from sklearn.linear_model import LinearRegression as skLinearRegression

In [2]:
n_samples = 2**20 
n_features = 399

random_state = 23

In [3]:
%%time
X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=random_state)

X = cudf.DataFrame(X)
y = cudf.DataFrame(y)[0]

X_cudf, X_cudf_test, y_cudf, y_cudf_test = train_test_split(X, y, test_size = 0.2, random_state=random_state)

CPU times: user 2.96 s, sys: 1.53 s, total: 4.49 s
Wall time: 21.5 s


In [4]:
%%time
X_train = X_cudf.to_pandas()
X_test = X_cudf_test.to_pandas()
y_train = y_cudf.to_pandas()
y_test = y_cudf_test.to_pandas()

CPU times: user 1.01 s, sys: 1.62 s, total: 2.63 s
Wall time: 2.62 s


## sklearn model


In [5]:
%%timeit
sk = skLinearRegression(fit_intercept=True,
                            normalize=True,
                            n_jobs=-1)

sk.fit(X_train, y_train)

1 loop, best of 5: 16.9 s per loop


In [8]:
sk = skLinearRegression(fit_intercept=True,
                            normalize=True,
                            n_jobs=-1)

sk.fit(X_train, y_train)

LinearRegression(n_jobs=-1, normalize=True)

In [9]:
%%time
predict_sk = sk.predict(X_test)

CPU times: user 126 ms, sys: 155 µs, total: 126 ms
Wall time: 102 ms


In [10]:
%%time
r2_score_sk = r2_score(y_test, predict_sk)

CPU times: user 4.56 ms, sys: 2.07 ms, total: 6.64 ms
Wall time: 15.4 ms


## GPU model

In [11]:
%%timeit
gpuModel = cuLinearRegression(fit_intercept=True,
                              normalize=True,
                              algorithm='eig')

gpuModel.fit(X_cudf, y_cudf)

1 loop, best of 5: 323 ms per loop


In [12]:
gpuModel = cuLinearRegression(fit_intercept=True,
                              normalize=True,
                              algorithm='eig')

gpuModel.fit(X_cudf, y_cudf)

LinearRegression()

In [13]:
%%time
predict_cuml = gpuModel.predict(X_cudf_test)

CPU times: user 25.7 ms, sys: 5.89 ms, total: 31.6 ms
Wall time: 39.7 ms


In [14]:
%%time
r2_score_cuml = r2_score(y_cudf_test, predict_cuml)

CPU times: user 1.06 ms, sys: 1 ms, total: 2.06 ms
Wall time: 2.07 ms


In [15]:
print("R^2 score (SKL):  %s" % r2_score_sk)
print("R^2 score (cuML): %s" % r2_score_cuml)

R^2 score (SKL):  1.0
R^2 score (cuML): 1.0


## General operations

In [16]:
import cudf 
import pandas as pd
import numpy as np

In [17]:
import warnings

warnings.filterwarnings("ignore")

In [18]:
%%timeit
gpuDF = cudf.DataFrame({'col_1': np.random.randint(0, 10000000, size=10000000),
                          'col_2': np.random.randint(0, 10000000, size=10000000)})

1 loop, best of 5: 320 ms per loop


In [19]:
%%timeit
pandasDF = pd.DataFrame({'col_1': np.random.randint(0, 10000000, size=10000000),
                          'col_2': np.random.randint(0, 10000000, size=10000000)})

1 loop, best of 5: 335 ms per loop


In [20]:
gpuDF = cudf.DataFrame({'col_1': np.random.randint(0, 100000000, size=100000000),
                          'col_2': np.random.randint(0, 100000000, size=100000000)})

In [21]:
pandasDF = pd.DataFrame({'col_1': np.random.randint(0, 100000000, size=100000000),
                          'col_2': np.random.randint(0, 100000000, size=100000000)})

### log

In [22]:
%%timeit -n 10

pandasDF['log_1'] = np.log(pandasDF['col_1'])

10 loops, best of 5: 2.64 s per loop


In [39]:
%%timeit -n 10

gpuDF['log_1'] = np.log(gpuDF['col_1'])

10 loops, best of 5: 33.7 ms per loop


### square

In [23]:
%%timeit -n 10

pandasDF['sqaure_1'] = np.sqrt(pandasDF['col_1'])

10 loops, best of 5: 547 ms per loop


In [24]:
%%timeit -n 10

gpuDF['square_1'] = np.sqrt(gpuDF['col_1'])

10 loops, best of 5: 15.3 ms per loop


### string conversion

In [ ]:
%%time 

pandasDF.col_1.astype(str)

In [14]:
%%time

gpuDF.col_1.astype(str)

CPU times: user 34.1 ms, sys: 3.76 ms, total: 37.9 ms
Wall time: 38 ms


0            3825623
1           22388050
2           96850930
3           70175343
4           63431439
              ...   
99999995    33588499
99999996    51181494
99999997    91417594
99999998    60706307
99999999    91123475
Name: col_1, Length: 100000000, dtype: object

In [15]:
!nvidia-smi

Mon Jun 21 17:59:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    34W /  70W |   6492MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------